# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N
68816,16000.0,0.1131,350.36,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,n,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N


## Split the Data into Training and Testing

In [5]:
df_binary_encoded = pd.get_dummies(df, columns=['home_ownership', 'issue_d','verification_status', 'pymnt_plan',
                                                'initial_list_status', 'next_pymnt_d','application_type','hardship_flag',
                                               'debt_settlement_flag'])
df_binary_encoded
# Create our features
X = df_binary_encoded.drop(columns='loan_status')

# Create our target
y = df_binary_encoded['loan_status']
display(X)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,1,1,1,0,0,1,1,0,1,1


In [6]:
X.describe()


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=64, random_state=40)
brf_model = brf_model.fit(X_train, y_train)
predictions = brf_model.predict(X_test)

In [13]:
# Calculated the balanced accuracy score
bal_acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Balanced Accuracy Score: {bal_acc_score}")

Balanced Accuracy Score: 0.8064257829024499


In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index =["Actual High Risk", "Actual Low Risk"],
    columns=["Predicted High Risk", "Predicted Low Risk"]
)
display(cm_df)

,Predicted High Risk,Predicted Low Risk
Actual High Risk,63,24
Actual Low Risk,1905,15213


In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.72      0.89      0.06      0.80      0.63        87
   low_risk       1.00      0.89      0.72      0.94      0.80      0.65     17118

avg / total       0.99      0.89      0.72      0.94      0.80      0.65     17205



In [16]:
# List the features sorted in descending order by feature importance
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.08007298342368584, 'total_rec_prncp'),
 (0.06879648536402826, 'total_pymnt'),
 (0.05930513767985236, 'last_pymnt_amnt'),
 (0.05606126577641204, 'total_pymnt_inv'),
 (0.05286050485032972, 'total_rec_int'),
 (0.0330172819956132, 'int_rate'),
 (0.02320638229511875, 'issue_d_Jan-2019'),
 (0.01955865095399103, 'out_prncp_inv'),
 (0.01750617133488982, 'installment'),
 (0.017068915764577305, 'mths_since_recent_inq'),
 (0.016911699131345118, 'dti'),
 (0.016446550402339173, 'bc_util'),
 (0.016158520993447587, 'out_prncp'),
 (0.016146922740999912, 'avg_cur_bal'),
 (0.014156879187861786, 'max_bal_bc'),
 (0.01404982151006016, 'total_bal_ex_mort'),
 (0.01378874345344299, 'issue_d_Mar-2019'),
 (0.013559593243576338, 'total_rev_hi_lim'),
 (0.013151487406551483, 'total_bc_limit'),
 (0.012960935325690414, 'revol_bal'),
 (0.012497197959583775, 'annual_inc'),
 (0.0122584752311374, 'mo_sin_rcnt_rev_tl_op'),
 (0.01223818192622449, 'mo_sin_old_il_acct'),
 (0.012194008387269037, 'total_il_high_credit_lim

### Easy Ensemble Classifier

In [17]:
# Train the Classifier
rf_model = RandomForestClassifier(n_estimators=64, random_state=40)
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [18]:
# Calculated the balanced accuracy score
bal_acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Balanced Accuracy Score: {bal_acc_score}")

Balanced Accuracy Score: 0.6548803235956505


In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index =["Actual High Risk", "Actual Low Risk"],
    columns=["Predicted High Risk", "Predicted Low Risk"]
)
display(cm_df)

,Predicted High Risk,Predicted Low Risk
Actual High Risk,27,60
Actual Low Risk,10,17108


In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.73      0.31      1.00      0.44      0.56      0.29        87
   low_risk       1.00      1.00      0.31      1.00      0.56      0.33     17118

avg / total       1.00      1.00      0.31      1.00      0.56      0.33     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The model with the best balance accuracy score was the Balanced Random Forest Classifier

2. Which model had the best recall score?

    The model with the best recall score was the Balanced Random Forest Classifier

3. Which model had the best geometric mean score?

    The model with the best geometric mean score was the Balanced Random Forest Classifier

4. What are the top three features?

    The top three features are total_rec_prncp, last_pymnt_amnt, and total_rec_int.